# Colab的默认环境跟这里的默认环境有差异，需要安装额外的依赖

In [7]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 94 not upgraded.
Need to get 2129 kB of archives.
After this operation, 7662 kB of additional disk space will be used.
Get:1 http://mirrors.tuna.tsinghua.edu.cn/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2129 kB]
Fetched 2129 kB in 0s (5726 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
Selecting previously unselected package git-lfs.
(Reading database ... 52279 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...


In [9]:
!pip install wget
!pip install transformers
!pip install bitsandbytes
!pip install jax
!apt-get update
!apt-get install -y p7zip-full


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.9 MB/s eta 0:00:00a 0:00:01m
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 38.1 kB/s eta 0:00:00a 0:00:01
  Created wheel for jax: filename=jax-0.3.25-py3-none-any.whl size=1308495 sha256=386cd1ae3be8471249cf179b2cb08193e7f22d3bf79b784ec3667c646b3439a4
  Stored in directory: /root/.cache/pip/wheels/ab/b9/18/0f42bd0c697d360a5f3a4bdfd3fd88d0c9e1f1cfd86b5f8caf
Successfully built jax
Hit:1 http://mirrors.tuna.tsinghua.edu.cn/ubuntu bionic InRelease
Hit:2 https://deb.nodesource.com/node_14.x bionic InRelease                    
Hit:3 http://mirrors.tuna.tsinghua.edu.cn/ubuntu bionic-backports InRelease    
Hit:4 http://mirrors.tuna.tsinghua

In [ ]:
import os,sys
sys.path.append('../') # 把上级路径也加入到系统路径中，这样就能够找到func
from func.env import getOneClickDir

oneClickDir=getOneClickDir()
print(oneClickDir)

In [11]:
import os
import wget

# 在当前运行的ipynb文件所在的目录下创建临时文件夹temp
def create_temp_folder(temp_folder):
  if not os.path.exists(temp_folder):
    os.makedirs(temp_folder)

# 获取当前运行的 ipynb 文件所在的目录
cwd = os.getcwd()

# 在该目录下创建临时文件夹 temp
temp_folder = os.path.join(cwd, 'temp')

# 如果temp文件夹不存在，则创建文件夹
create_temp_folder(temp_folder)

if not os.path.exists(temp_folder):
    os.makedirs(temp_folder)

# 覆盖下载 fast-DreamBooth.ipynb 文件到 temp 文件夹中
file_name='fast-DreamBooth.ipynb'
fastDreamBoothPath = os.path.join(temp_folder, file_name)
if os.path.exists(fastDreamBoothPath):
    os.remove(fastDreamBoothPath)
fastDreamBoothPath = wget.download(
    "https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/fast-DreamBooth.ipynb",
    out=os.path.join(temp_folder, file_name)
)
print(fastDreamBoothPath)

/output/OneClick-stable-diffusion/dreambooth/temp/fast-DreamBooth.ipynb


# 清理Notebook中的输出，对Colab版中的路径进行替换，以适应openbayes的路径

In [23]:
import json
import os

def modify_file_name(file_path):
  # 获取文件名和文件扩展名
  file_name, file_ext = os.path.splitext(file_path)
  # 在文件名后加上 -modified
  modified_file_path = file_name + "-modified" + file_ext
  return modified_file_path

def replace_root_path(notebook: str, old_root: str, new_root: str) -> str:
    # 加载notebook
    with open(notebook, "r") as f:
        nb = json.load(f)

    # 隐藏代码
    for cell in nb["cells"]:
        if cell["cell_type"] == "code":
            cell["metadata"] = {"collapsed": True}

    # 清空notebook中所有输出
    for cell in nb['cells']:
        if cell['cell_type'] == 'code':
            # 清除代码单元格的执行次数
            cell['execution_count'] = None
            # 清除代码单元格的输出
            cell['outputs'] = []

    # 替换旧的根路径为新的根路径
    for cell in nb["cells"]:
        if cell["cell_type"] == "code":
            for i, line in enumerate(cell["source"]):
                # 添加一个条件判断，只有当该行不是网址路径时才进行替换
                if not line.startswith(("http", "https")):
                    cell["source"][i] = line.replace(old_root, new_root)
                # 把引入google.colab的库的代码行标注为注释
                if "google.colab" in line:
                    cell["source"][i] = "# " + line
                # 把使用google drive盘挂载的代码行标注为注释
                if "drive.mount" in line:
                    cell["source"][i] = "# " + line
                
                # # 原始字符串
                # original = "!cp -r /openbayes/home/content/usr/local/lib/python3.8/dist-packages /usr/local/lib/python3.8/"
                # if original in line:
                #     # 替换python路径
                #     cell["source"][i] = re.sub(r'/usr/local/lib/python3.8/', '/usr/local/lib/python/', original)
                    
    # 替换@param{type: 'xxx'}中的单引号为双引号
    for cell in nb["cells"]:
        if cell["cell_type"] == "code":
            for i, line in enumerate(cell["source"]):
                if "@param" in line:
                    cell["source"][i] = line.replace("'", '"')
                    
    # 保存修改后的notebook
    newNotebook=modify_file_name(notebook)
    with open(newNotebook, "w") as f:
        json.dump(nb, f)
        
    return newNotebook

newNotebookPath=replace_root_path(
    notebook=fastDreamBoothPath,
    old_root="/content",
    new_root="/openbayes/home/content"
)
print(newNotebookPath)

/output/OneClick-stable-diffusion/dreambooth/temp/fast-DreamBooth-modified.ipynb


# 对指定的变量名进行字符串和参数的替换

In [24]:
import json

def keepAnnotationBehide(line):
  identifier="#@param"
  if identifier in line:
    annotation=line.split(identifier)[1]
    return identifier+annotation

def modify_file_name(file_path):
  # 获取文件名和文件扩展名
  file_name, file_ext = os.path.splitext(file_path)
  # 在文件名后加上 -modified
  modified_file_path = file_name + "-customData" + file_ext
  return modified_file_path

# 读取修改后的notebook
def update_ipynb_vars(ipynb_file, var_map):
  with open(ipynb_file, 'r') as f:
    notebook = json.load(f)
  
  # 用更新后的值替换变量赋值
  for cell in notebook['cells']:
    if cell['cell_type'] == 'code':
      for i in range(len(cell['source'])):
        for var, val in var_map.items():
          if isinstance(val, str):
            val = f'"{val}"'
          if cell['source'][i].startswith(f'{var} = '):
            annotation=keepAnnotationBehide(cell['source'][i])
            cell['source'][i] = f'{var} = {val} {annotation}\n' 


  # 保存修改后的notebook
  updatedDataNotebookPath=modify_file_name(ipynb_file)
  with open(updatedDataNotebookPath, 'w') as f:
    json.dump(notebook, f)

  return updatedDataNotebookPath

# 更新变量
replacements = {
  'Model_Version': 'V2.1-768px',
  'Session_Name': 'FuXingHao768',
  'Crop_size': 768,
  'Remove_existing_instance_images': False,
  'IMAGES_FOLDER_OPTIONAL':"/openbayes/home/content/gdrive/MyDrive/Fast-Dreambooth/Sessions/FuXingHao768/instance_images",
  'Crop_images':False,
  'Resolution': 768,
}

# 更新变量
updatedDataNotebookPath=update_ipynb_vars(newNotebookPath, replacements)

# 检查最终版本的ipynb里面的赋值情况

In [14]:
import json

def detect_constant_assignments(ipynb_file: str):
    with open(ipynb_file, 'r') as f:
        ipynb_data = json.load(f)

    # 遍历所有的代码单元
    for cell in ipynb_data['cells']:
        if cell['cell_type'] == 'code':
            # 遍历当前代码单元中的所有行
            for line in cell['source']:
                # 检测是否是赋值行，并获取变量名和值
                parts = line.split('=')
                if len(parts) == 2:
                    var_name = parts[0].strip()
                    value = parts[1].strip()
                    print(f'{var_name}: {value}')

detect_constant_assignments(updatedDataNotebookPath)


Model_Version: "V2.1-768px" #@param [ "1.5", "V2.1-512px", "V2.1-768px"]
Huggingface_Token: "" #@param {type:"string"}
token: Huggingface_Token
Custom_Model_Version: "1.5" #@param [ "1.5", "V2.1-512px", "V2.1-768px"]
Path_to_HuggingFace: "" #@param {type:"string"}
CKPT_Path: "" #@param {type:"string"}
CKPT_Link: "" #@param {type:"string"}
token: Huggingface_Token
token: input("Insert your huggingface token :")
if Path_to_HuggingFace !: "":
MODEL_NAME: "/openbayes/home/content/stable-diffusion-custom"
MODEL_NAME: "/openbayes/home/content/stable-diffusion-custom"
elif CKPT_Path !: "":
MODEL_NAME: "/openbayes/home/content/stable-diffusion-custom"
elif CKPT_Link !: "":
MODEL_NAME: "/openbayes/home/content/stable-diffusion-custom"
MODEL_NAME: "/openbayes/home/content/stable-diffusion-v1-5"
MODEL_NAME: "/openbayes/home/content/stable-diffusion-v1-5"
MODEL_NAME: "/openbayes/home/content/stable-diffusion-v2-512"
MODEL_NAME: "/openbayes/home/content/stable-diffusion-v2-512"
MODEL_NAME: "/openba